# 주관식문제 1. 메소드 convert_examples_to_features에서 line 37의 (a)와 line 78~86 (a), (b), (c)를 채워 input data를 preprocessing하는 코드를 완성한 후 jupyter notebook의 모든 셀을 실행하시오. (모든 빈칸 (a)는 같은 변수가 들어갑니다.)

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import collections
import logging
import json
import math
import os
import random
import pickle
import numpy as np
import torch

from tqdm import tqdm, trange
from bert_utils.bert_utils import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import whitespace_tokenize, BasicTokenizer, BertTokenizer
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer."""
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)
    return (input_start, input_end)
def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index


Configuration settings

In [7]:
train_file = './datasets/korquad_v1/KorQuAD_v1.0_train.json'
dev_file = './datasets/korquad_v1/KorQuAD_v1.0_dev.json'    

In [8]:
def read_json(file_name):
    print("reading {}".format(file_name))
    with open(file_name, "r", encoding='utf-8') as reader:
        # >> type(input_data) -> list
        input_data = json.load(reader)["data"]
    print("success to read {}".format(file_name))
    return input_data

In [9]:
input_data = read_json(train_file)

reading ./datasets/korquad_v1/KorQuAD_v1.0_train.json
success to read ./datasets/korquad_v1/KorQuAD_v1.0_train.json


explore the datasets

## parse datasets

In [10]:
class SquadExample(object):
    """A single training/test example for the Squad dataset."""

    def __init__(self,
                 qas_id,
                 question_text,
                 doc_tokens,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None):
        self.qas_id = qas_id
        self.question_text = question_text
        self.doc_tokens = doc_tokens
        self.orig_answer_text = orig_answer_text
        self.start_position = start_position
        self.end_position = end_position

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += "\n\n, question_text: %s" % (
            self.question_text)
        s += "\n\n, orig_answer_text: %s" % (
            self.orig_answer_text)
        s += "\n\n, doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        if self.start_position:
            s += "\n\n, start_position: %d" % (self.start_position)
        if self.start_position:
            s += "\n\n, end_position: %d" % (self.end_position)
        return s

In [ ]:
#실습 자료

In [11]:
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

def parse_json_squad(input_data, is_train):
    """Read a SQuAD json file into a list of SquadExample."""
    examples = list()
    for data_entry in input_data:
        for paragraph in data_entry['paragraphs']:
            paragraph_text = paragraph["context"]
            doc_tokens = []
            char_to_word_offset = []
            prev_is_whitespace = True

            for char in paragraph_text:
                if is_whitespace(char):
                    prev_is_whitespace = True
                else:
                    if prev_is_whitespace:
                        doc_tokens.append(char)
                    else:
                        doc_tokens[-1] += char
                    prev_is_whitespace = False
                char_to_word_offset.append(len(doc_tokens) - 1)  # Which word is the character in?
    
            for qa in paragraph["qas"]:
                """
                {'answers': [{'answer_start', 'text'}], 'question', 'id'}
                """
                qas_id = qa["id"]
                question_text = qa["question"]
                start_position = None
                end_position = None
                orig_answer_text = None

                if is_train:
                    if len(qa["answers"]) != 1:
                        raise ValueError(
                            "For training, each question should have exactly 1 answer.")                    
                    qas_id = qa["id"]     # fill the black -> assign None
                    question_text = qa["question"] # fill the black # index of word
                    answer = qa["answers"][0]
                    orig_answer_text = answer["text"]
                    answer_offset = answer["answer_start"]
                    answer_length = len(orig_answer_text)
                    start_position = char_to_word_offset[answer_offset] # index of word
                    end_position = char_to_word_offset[answer_offset + answer_length - 1] # index of word
                    actual_text = " ".join(doc_tokens[start_position:(end_position + 1)])
                    cleaned_answer_text = " ".join(whitespace_tokenize(
                        orig_answer_text))  # segment words from the sentense including the white space
                    if actual_text.find(cleaned_answer_text) == -1:
                        logger.warning("Could not find answer: '%s' vs. '%s'",
                                       actual_text, cleaned_answer_text)
                        continue

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    doc_tokens=doc_tokens,  # a set of tokens(words) in the
                    orig_answer_text=orig_answer_text,
                    start_position=start_position,
                    end_position=end_position)
                examples.append(example)
    print("success to convert input data into a set of {} examples".format(len(examples)))
    return examples

In [12]:
train_examples = parse_json_squad(input_data, True)

success to convert input data into a set of 60407 examples


In [13]:
print(train_examples[0])

qas_id: 6566495-0-0

, question_text: 바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?

, orig_answer_text: 교향곡

, doc_tokens: [1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.]

, start_position: 14

, end_position: 14


## Extract Feature

In [14]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 start_position=None,
                 end_position=None):
        
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.start_position = start_position
        self.end_position = end_position

In [15]:
def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length, is_training):
    """Loads a data file into a list of `InputBatch`s."""

    unique_id = 1000000000

    features = []
    examples = examples[:20]
    for (example_index, example) in enumerate(examples):
        query_tokens = tokenizer.tokenize(example.question_text)
        if len(query_tokens) > max_query_length:
            query_tokens = query_tokens[0:max_query_length]
            
        tok_to_orig_index = []
        orig_to_tok_index = []
        all_doc_tokens = []     
        for (i, token) in enumerate(example.doc_tokens):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens: 
                tok_to_orig_index.append(i)                  
                all_doc_tokens.append(sub_token)
                
        tok_start_position = None
        tok_end_position = None
        if is_training:
            tok_start_position = orig_to_tok_index[example.start_position]
            if example.end_position < len(example.doc_tokens) - 1:
                tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
            else:
                tok_end_position = len(all_doc_tokens) - 1
            (tok_start_position, tok_end_position) = _improve_answer_span(
                all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
                example.orig_answer_text)
        
        # 1. (a)를 채우시오
        max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset # 남은 길이
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)
            
            
        for (doc_span_index, doc_span) in enumerate(doc_spans):
            tokens = [] # input data
            segment_ids = [] # segment data
            token_to_orig_map = {}
            token_is_max_context = {}
            tokens.append("[CLS]")
            segment_ids.append(0)
            for token in query_tokens:
                tokens.append(token)
                segment_ids.append(0)
            tokens.append("[SEP]")
            segment_ids.append(0)

            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]
                is_max_context = _check_is_max_context(doc_spans,doc_span_index,split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(1) # segment ids 12 means the context
            tokens.append("[SEP]")
            segment_ids.append(1)
            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            
            # 2. (a), (b), (c)를 채우시오 2.(a)는 1.(a)와 같습니다.
            input_mask = [1] * len(input_ids)
            while len(input_ids) < max_seq_length :
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length
            ###################################################################################################
            
            start_position = None
            end_position = None
            if is_training:
                # For training, if our document chunk does not contain an annotation
                # we throw it out, since there is nothing to predict.
                doc_start = doc_span.start
                doc_end = doc_span.start + doc_span.length - 1
                if (example.start_position < doc_start or
                        example.end_position < doc_start or
                        example.start_position > doc_end or example.end_position > doc_end):
                    continue # -> next to the DocSpan 

                doc_offset = len(query_tokens) + 2
                start_position = tok_start_position - doc_start + doc_offset
                end_position = tok_end_position - doc_start + doc_offset

            if example_index < 20:
                logger.info("*** Example ***")
                logger.info("unique_id: %s" % (unique_id))
                logger.info("example_index: %s" % (example_index))
                logger.info("doc_span_index: %s" % (doc_span_index))
                logger.info("tokens: %s" % " ".join(tokens))
                logger.info("token_to_orig_map: %s" % " ".join([
                    "%d:%d" % (x, y) for (x, y) in token_to_orig_map.items()]))
                logger.info("token_is_max_context: %s" % " ".join([
                    "%d:%s" % (x, y) for (x, y) in token_is_max_context.items()
                ]))
                logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
                logger.info(
                    "input_mask: %s" % " ".join([str(x) for x in input_mask]))
                logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
                if is_training:
                    answer_text = " ".join(tokens[start_position:(end_position + 1)])
                    logger.info("start_position: %d" % (start_position))
                    logger.info("end_position: %d" % (end_position))
                    logger.info(
                        "answer: %s" % (answer_text))
                    
            features.append(
                InputFeatures(
                    unique_id=unique_id,
                    example_index=example_index,
                    doc_span_index=doc_span_index,
                    tokens=tokens,
                    token_to_orig_map=token_to_orig_map,
                    token_is_max_context=token_is_max_context,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    start_position=start_position,
                    end_position=end_position))
            unique_id += 1

    return features

In [16]:
config='bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(config)
max_query_length=64
max_seq_length=128
doc_stride=128
max_query_length=64

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

train_features = convert_examples_to_features(
    examples=train_examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=True)

print('finish extracting the features from the examples')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 995526/995526 [00:01<00:00, 716663.47B/s]
08/11/2020 15:22:52 - INFO - __main__ -   *** Example ***
08/11/2020 15:22:52 - INFO - __main__ -   unique_id: 1000000000
08/11/2020 15:22:52 - INFO - __main__ -   example_index: 0
08/11/2020 15:22:52 - INFO - __main__ -   doc_span_index: 0
08/11/2020 15:22:52 - INFO - __main__ -   tokens: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽 ##고 무 ##엇 ##을 쓰 ##고 ##자 했 ##는 ##가 ? [SEP] 1839 ##년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽 ##고 그 내 ##용 ##에 마 ##음 ##이 끌 ##려 이를 소 ##재 ##로 해 ##서 하나의 교 ##향 ##곡 ##을 쓰 ##려 ##는 뜻 ##을 갖 ##는다 . 이 시 ##기 바 ##그 ##너 ##는 1838 ##년에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 [UNK] 상 ##황 ##이 ##라 좌 ##절 ##과 실 ##망 ##에 가 ##득 ##했으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만 ##나는 파 ##우스 ##트 ##의 심 ##경 ##에 공 ##감 ##했다 ##고 한다 . 또한 파 ##리에 ##서 아 [SEP]
08/11/2020 15:

08/11/2020 15:22:52 - INFO - __main__ -   example_index: 2
08/11/2020 15:22:52 - INFO - __main__ -   doc_span_index: 0
08/11/2020 15:22:52 - INFO - __main__ -   tokens: [CLS] 바 ##그 ##너 ##가 파 ##우스 ##트 서 ##곡 ##을 쓸 때 어떤 곡 ##의 영향을 받 ##았 ##는 ##가 ? [SEP] 1839 ##년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽 ##고 그 내 ##용 ##에 마 ##음 ##이 끌 ##려 이를 소 ##재 ##로 해 ##서 하나의 교 ##향 ##곡 ##을 쓰 ##려 ##는 뜻 ##을 갖 ##는다 . 이 시 ##기 바 ##그 ##너 ##는 1838 ##년에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 [UNK] 상 ##황 ##이 ##라 좌 ##절 ##과 실 ##망 ##에 가 ##득 ##했으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만 ##나는 파 ##우스 ##트 ##의 심 ##경 ##에 공 ##감 ##했다 ##고 한다 . 또한 파 ##리에 ##서 아 ##브 [SEP]
08/11/2020 15:22:52 - INFO - __main__ -   token_to_orig_map: 23:0 24:0 25:1 26:1 27:1 28:1 29:2 30:2 31:2 32:3 33:3 34:3 35:3 36:4 37:5 38:5 39:6 40:7 41:7 42:7 43:8 44:8 45:8 46:9 47:9 48:10 49:11 50:11 51:11 52:12 53:12 54:13 55:14 56:14 57:14 58:14 59:15 60:15 61:15 62:16 63:16 64:17 65:17 66:17 67:18 68:19 69:19 70:20 71:20 72:20 73:20 74:21 75:21 76:22 77:23 78:23 79:23 80:24 8

08/11/2020 15:22:53 - INFO - __main__ -   token_to_orig_map: 22:0 23:0 24:1 25:1 26:1 27:1 28:2 29:2 30:2 31:3 32:3 33:3 34:3 35:4 36:5 37:5 38:6 39:7 40:7 41:7 42:8 43:8 44:8 45:9 46:9 47:10 48:11 49:11 50:11 51:12 52:12 53:13 54:14 55:14 56:14 57:14 58:15 59:15 60:15 61:16 62:16 63:17 64:17 65:17 66:18 67:19 68:19 69:20 70:20 71:20 72:20 73:21 74:21 75:22 76:23 77:23 78:23 79:24 80:24 81:24 82:24 83:25 84:26 85:27 86:27 87:27 88:27 89:28 90:28 91:28 92:29 93:29 94:29 95:30 96:30 97:30 98:31 99:31 100:31 101:31 102:31 103:31 104:31 105:32 106:32 107:33 108:33 109:33 110:33 111:34 112:34 113:34 114:35 115:35 116:35 117:35 118:36 119:36 120:37 121:38 122:38 123:38 124:39 125:39 126:39
08/11/2020 15:22:53 - INFO - __main__ -   token_is_max_context: 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:

08/11/2020 15:22:53 - INFO - __main__ -   token_is_max_context: 17:True 18:True 19:True 20:True 21:True 22:True 23:True 24:True 25:True 26:True 27:True 28:True 29:True 30:True 31:True 32:True 33:True 34:True 35:True 36:True 37:True 38:True 39:True 40:True 41:True 42:True 43:True 44:True 45:True 46:True 47:True 48:True 49:True 50:True 51:True 52:True 53:True 54:True 55:True 56:True 57:True 58:True 59:True 60:True 61:True 62:True 63:True 64:True 65:True 66:True 67:True 68:True 69:True 70:True 71:True 72:True 73:True 74:True 75:True 76:True 77:True 78:True 79:True 80:True 81:True 82:True 83:True 84:True 85:True 86:True 87:True 88:True 89:True 90:True 91:True 92:True 93:True 94:True 95:True 96:True 97:True 98:True 99:True 100:True 101:True 102:True 103:True 104:True 105:True 106:True 107:True 108:True 109:True 110:True 111:True 112:True 113:True 114:True 115:True 116:True 117:True 118:True 119:True 120:True 121:True 122:True 123:True 124:True 125:True 126:True
08/11/2020 15:22:53 - INFO - 

08/11/2020 15:22:53 - INFO - __main__ -   input_ids: 101 9318 78136 70162 10459 9652 104485 9485 48418 9946 11261 9511 29935 32855 11664 8925 119249 11102 30050 9032 17196 12030 11287 136 102 53519 14701 87188 9318 78136 70162 12638 9524 11664 9706 31605 23990 9238 34994 11287 9649 80579 41584 43646 122 119110 35963 9365 119446 14040 100110 15307 27056 9318 10739 23811 31401 11489 9568 16323 12490 119 9638 93834 8887 62675 9318 78136 70162 12092 9638 9652 52951 10530 25805 8900 86904 8854 14153 81984 117 8924 9960 16959 15303 9761 30005 10459 9321 51745 10622 9599 17196 13374 35756 9425 55670 11467 8845 119274 27654 52292 9460 40818 9965 11664 9376 17342 41620 25517 28396 119460 111 9973 31401 119354 9768 33323 86580 12424 9768 33323 14843 8857 16605 33323 12092 102
08/11/2020 15:22:53 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

...

08/11/2020 15:22:53 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
08/11/2020 15:22:53 - INFO - __main__ -   start_position: 259
08/11/2020 15:22:53 - INFO - __main__ -   end_position: 262
08/11/2020 15:22:53 - INFO - __main__ -   answer: 
08/11/2020 15:22:53 - INFO - __main__ -   *** Example ***
08/11/2020 15:22:53 - INFO - __main__ -   unique_id: 1000000013
08/11/2020 15:22:53 - INFO - __main__ -   example_index: 13
08/11/2020 15:22:53 - INFO - __main__ -   doc_span_index: 0
08/11/2020 15:22:53 - INFO - __main__ -   tokens: [CLS] 1 ##악 ##장을 부 ##활 ##시 ##켜 연 ##주 ##한 사 ##람 ##은 ? [SEP] 한편 1840 ##년부터 바 ##그 ##너 ##와 알 ##고 지 ##내 ##던 리 ##스트 ##가 잊 ##혀 ##져 있던 1 ##악 ##장을 부 ##활 ##시 ##켜 1852 ##년에 바 ##이 ##마 ##르 ##에서 연 ##주 ##했다 . 이 ##것을 계 ##기로 바 ##그 ##너 ##도 이 

08/11/2020 15:22:53 - INFO - __main__ -   start_position: 260
08/11/2020 15:22:53 - INFO - __main__ -   end_position: 263
08/11/2020 15:22:53 - INFO - __main__ -   answer: 
08/11/2020 15:22:53 - INFO - __main__ -   *** Example ***
08/11/2020 15:22:53 - INFO - __main__ -   unique_id: 1000000015
08/11/2020 15:22:53 - INFO - __main__ -   example_index: 15
08/11/2020 15:22:53 - INFO - __main__ -   doc_span_index: 0
08/11/2020 15:22:53 - INFO - __main__ -   tokens: [CLS] 리 ##스트 ##가 바 ##그 ##너 ##와 알 ##게 된 연 ##도는 ? [SEP] 한편 1840 ##년부터 바 ##그 ##너 ##와 알 ##고 지 ##내 ##던 리 ##스트 ##가 잊 ##혀 ##져 있던 1 ##악 ##장을 부 ##활 ##시 ##켜 1852 ##년에 바 ##이 ##마 ##르 ##에서 연 ##주 ##했다 . 이 ##것을 계 ##기로 바 ##그 ##너 ##도 이 작 ##품 ##에 다시 관 ##심을 갖 ##게 되었고 , 그 해 9월 ##에는 총 ##보 ##의 반 ##환 ##을 요 ##구 ##하여 이를 서 ##곡 ##으로 간 ##추 ##린 다음 수 ##정을 했 ##고 브 ##라 ##이트 ##코 ##프 ##흐 & 헤 ##르 ##텔 출 ##판 ##사에 ##서 출 ##판 ##할 개 ##정 ##판 ##도 준 ##비 ##했다 . 1853 ##년 5월 ##에는 리 ##스트 [SEP]
08/11/2020 15:22:53 - INFO - __main__ -   token_to_orig_map: 15:0 16:1 17:1 18:2 19:

08/11/2020 15:22:53 - INFO - __main__ -   *** Example ***
08/11/2020 15:22:53 - INFO - __main__ -   unique_id: 1000000017
08/11/2020 15:22:53 - INFO - __main__ -   example_index: 17
08/11/2020 15:22:53 - INFO - __main__ -   doc_span_index: 0
08/11/2020 15:22:53 - INFO - __main__ -   tokens: [CLS] 첫 ##부 ##분에 ##는 어떤 악 ##기를 사 ##용 ##해 더욱 명 ##확 ##하게 나 ##타 ##내는 ##가 ? [SEP] 이 작 ##품 ##은 라 ##단 ##조 , Se ##hr gehalten ( 아 ##주 신 ##중 ##하게 ) , 4 / 4 ##박 ##자의 부 ##드 ##러 ##운 서 ##주 ##로 서 ##주 ##로 시 ##작 ##되는 ##데 , 여 ##기에 ##는 주요 주 ##제 , 동 ##기의 대부분 ##이 암 ##시 , 예 ##고 ##되어 있다 . 첫 부 ##분 ##의 저 ##음 주 ##제는 주요 주 ##제 ( 고 ##뇌 ##와 갈 ##망 동 ##기 , 청 ##춘 ##의 사 ##랑 동 ##기 ) 를 암 ##시 ##하고 있으며 , 제 ##1 ##바 ##이 ##올 ##린 ##으로 더욱 명 ##확 ##하게 나 ##타 ##난 [SEP]
08/11/2020 15:22:53 - INFO - __main__ -   token_to_orig_map: 21:0 22:1 23:1 24:1 25:2 26:2 27:2 28:2 29:3 30:3 31:4 32:4 33:4 34:4 35:5 36:5 37:5 38:5 39:5 40:6 41:6 42:6 43:6 44:6 45:7 46:7 47:7 48:7 49:8 50:8 51:8 52:9 53:9 54:9 55:10 56:10 57:10 58:10 59:10 60:11 61:11 62:11 

08/11/2020 15:22:53 - INFO - __main__ -   tokens: [CLS] 첫 부 ##분 ##의 주요 ##주 ##제를 암 ##시 ##하는 주 ##제는 ? [SEP] 이 작 ##품 ##은 라 ##단 ##조 , Se ##hr gehalten ( 아 ##주 신 ##중 ##하게 ) , 4 / 4 ##박 ##자의 부 ##드 ##러 ##운 서 ##주 ##로 서 ##주 ##로 시 ##작 ##되는 ##데 , 여 ##기에 ##는 주요 주 ##제 , 동 ##기의 대부분 ##이 암 ##시 , 예 ##고 ##되어 있다 . 첫 부 ##분 ##의 저 ##음 주 ##제는 주요 주 ##제 ( 고 ##뇌 ##와 갈 ##망 동 ##기 , 청 ##춘 ##의 사 ##랑 동 ##기 ) 를 암 ##시 ##하고 있으며 , 제 ##1 ##바 ##이 ##올 ##린 ##으로 더욱 명 ##확 ##하게 나 ##타 ##난 ##다 . 또한 그 ##것을 이어 [SEP]
08/11/2020 15:22:53 - INFO - __main__ -   token_to_orig_map: 15:0 16:1 17:1 18:1 19:2 20:2 21:2 22:2 23:3 24:3 25:4 26:4 27:4 28:4 29:5 30:5 31:5 32:5 33:5 34:6 35:6 36:6 37:6 38:6 39:7 40:7 41:7 42:7 43:8 44:8 45:8 46:9 47:9 48:9 49:10 50:10 51:10 52:10 53:10 54:11 55:11 56:11 57:12 58:13 59:13 60:13 61:14 62:14 63:15 64:15 65:16 66:16 67:16 68:17 69:17 70:17 71:18 72:18 73:19 74:20 75:20 76:20 77:21 78:21 79:22 80:22 81:23 82:24 83:24 84:24 85:24 86:24 87:24 88:25 89:25 90:26 91:26 92:26 93:27 94:27 95:27 96:28 97:28

finish extracting the features from the examples
